# **Chapter 44: Project 1 - E-Commerce App**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Architect a production-grade Flutter application using Clean Architecture principles
- Implement the BLoC pattern for scalable state management across complex features
- Build a performant product catalog with pagination, search, and filtering
- Design a shopping cart with optimistic updates and local persistence
- Integrate payment gateways (Stripe/PayPal) with secure tokenization
- Implement order tracking with real-time updates via push notifications
- Configure dependency injection for testable, maintainable code
- Handle error states and loading states gracefully throughout the user journey

---

## **Prerequisites**

- Completed Chapters 12-14: State Management (Provider, Riverpod, BLoC)
- Completed Chapter 19: HTTP Requests & REST APIs
- Completed Chapter 22: Local Data Persistence (Hive/SQLite)
- Completed Chapter 42: Security Best Practices (for payment handling)
- Completed Chapter 43: Build & Release (for deployment considerations)
- Understanding of RESTful API design and JSON serialization
- Basic knowledge of payment processing concepts (PCI compliance basics)

---

## **44.1 Architecture Overview**

We will implement **Clean Architecture** with three distinct layers: Presentation, Domain, and Data. This ensures business logic is independent of frameworks and UI.

### **Project Structure**

```
lib/
├── main.dart
├── injection.dart              # Dependency injection configuration
├── config/                     # App configuration, themes, routes
│   ├── routes.dart
│   └── theme.dart
├── core/                       # Shared utilities, errors, usecases
│   ├── errors/
│   ├── usecases/
│   └── utils/
├── features/                   # Feature-based modules
│   ├── product/               # Product catalog feature
│   │   ├── data/
│   │   ├── domain/
│   │   └── presentation/
│   ├── cart/                  # Shopping cart feature
│   │   ├── data/
│   │   ├── domain/
│   │   └── presentation/
│   ├── checkout/              # Payment & checkout
│   │   ├── data/
│   │   ├── domain/
│   │   └── presentation/
│   └── order/                 # Order tracking
│       ├── data/
│       ├── domain/
│       └── presentation/
└── services/                  # Shared services (DI, Navigation)
```

**Explanation:**

- **Clean Architecture**: Inner layers (Domain) don't depend on outer layers (Data/Presentation). Dependencies point inward.
- **Feature-First Organization**: Code organized by feature rather than layer, making it easier to scale and maintain.
- **Domain Layer**: Contains business logic (entities, use cases) independent of Flutter or external libraries.
- **Data Layer**: Implements repositories defined in Domain, handling API calls and local storage.
- **Presentation Layer**: UI widgets and state management (BLoCs) that coordinate with Use Cases.

---

## **44.2 Dependency Injection Configuration**

Using `get_it` and `injectable` for compile-time dependency injection.

```dart
// lib/injection.dart
import 'package:get_it/get_it.dart';
import 'package:injectable/injectable.dart';
import 'injection.config.dart';

final getIt = GetIt.instance;

@InjectableInit(
  initializerName: 'init', // default
  preferRelativeImports: true,
  asExtension: true,
)
void configureDependencies() => getIt.init();
```

```dart
// lib/features/product/data/repositories/product_repository_impl.dart
import 'package:injectable/injectable.dart';

@LazySingleton(as: ProductRepository)
class ProductRepositoryImpl implements ProductRepository {
  final ProductRemoteDataSource remoteDataSource;
  final ProductLocalDataSource localDataSource;
  final NetworkInfo networkInfo;

  ProductRepositoryImpl({
    required this.remoteDataSource,
    required this.localDataSource,
    required this.networkInfo,
  });
  
  // Implementation...
}
```

```yaml
# pubspec.yaml
dependencies:
  flutter:
    sdk: flutter
  # Architecture
  flutter_bloc: ^8.1.3
  get_it: ^7.6.4
  injectable: ^2.3.2
  freezed_annotation: ^2.4.1
  
  # Networking
  dio: ^5.4.0
  retrofit: ^4.0.3
  
  # Local Storage
  hive: ^2.2.3
  hive_flutter: ^1.1.0
  
  # Payment
  flutter_stripe: ^9.5.0+1
  
dev_dependencies:
  build_runner: ^2.4.8
  injectable_generator: ^2.4.1
  retrofit_generator: ^8.0.6
  hive_generator: ^2.0.1
  freezed: ^2.4.6
```

**Explanation:**

- **get_it**: Service locator pattern for dependency injection. `LazySingleton` creates instances on first use.
- **injectable**: Code generation for DI configuration. Reduces boilerplate in `injection.dart`.
- **Retrofit**: Type-safe HTTP client generator (similar to Android's Retrofit). Generates API clients from abstract classes.
- **Freezed**: Immutable data classes with copy methods, used for BLoC states and API models.

---

## **44.3 Domain Layer: Entities and Use Cases**

Defining business logic independent of implementation details.

```dart
// lib/features/product/domain/entities/product.dart
import 'package:freezed_annotation/freezed_annotation.dart';

part 'product.freezed.dart';
part 'product.g.dart';

@freezed
class Product with _$Product {
  const factory Product({
    required String id,
    required String name,
    required String description,
    required double price,
    required String currency,
    required List<String> images,
    required String category,
    required double rating,
    required int reviewCount,
    required bool inStock,
    required Map<String, dynamic> attributes, // Size, color, etc.
  }) = _Product;

  factory Product.fromJson(Map<String, dynamic> json) => 
      _$ProductFromJson(json);
}

// Value object for price to handle currency formatting
@freezed
class Price with _$Price {
  const factory Price({
    required double amount,
    required String currency,
  }) = _Price;

  const Price._();

  String get formatted {
    switch (currency) {
      case 'USD':
        return '\$${amount.toStringAsFixed(2)}';
      case 'EUR':
        return '€${amount.toStringAsFixed(2)}';
      default:
        return '$amount $currency';
    }
  }
}
```

**Explanation:**

- **Freezed**: Generates immutable classes with `copyWith`, `toString`, `==`, and JSON serialization.
- **Value Objects**: `Price` encapsulates formatting logic. Domain entities should contain business rules.
- **Attributes Map**: Flexible schema for product variations (size, color, specifications) without rigid typing.

```dart
// lib/features/product/domain/repositories/product_repository.dart
abstract class ProductRepository {
  /// Get paginated products with optional filters
  Future<Either<Failure, PaginatedResponse<Product>>> getProducts({
    required int page,
    required int limit,
    String? category,
    String? searchQuery,
    double? minPrice,
    double? maxPrice,
    String? sortBy,
  });

  /// Get single product details
  Future<Either<Failure, Product>> getProductDetails(String id);

  /// Get product categories
  Future<Either<Failure, List<Category>>> getCategories();
}

// lib/features/product/domain/usecases/get_products.dart
@injectable
class GetProducts implements UseCase<PaginatedResponse<Product>, ProductParams> {
  final ProductRepository repository;

  GetProducts(this.repository);

  @override
  Future<Either<Failure, PaginatedResponse<Product>>> call(ProductParams params) async {
    return await repository.getProducts(
      page: params.page,
      limit: params.limit,
      category: params.category,
      searchQuery: params.searchQuery,
      minPrice: params.minPrice,
      maxPrice: params.maxPrice,
      sortBy: params.sortBy,
    );
  }
}

class ProductParams {
  final int page;
  final int limit;
  final String? category;
  final String? searchQuery;
  final double? minPrice;
  final double? maxPrice;
  final String? sortBy;

  ProductParams({
    this.page = 1,
    this.limit = 20,
    this.category,
    this.searchQuery,
    this.minPrice,
    this.maxPrice,
    this.sortBy,
  });
}
```

**Explanation:**

- **Either Type**: Functional programming pattern for error handling. `Either<Failure, Success>` forces explicit error handling.
- **UseCase Pattern**: Each business operation is encapsulated in a use case class. This makes business logic reusable and testable.
- **Repository Pattern**: Abstract repository in Domain, implemented in Data layer. Domain doesn't know about HTTP or databases.

---

## **44.4 Data Layer: API Integration**

Implementing the repository with Retrofit and Dio.

```dart
// lib/features/product/data/datasources/product_remote_datasource.dart
import 'package:retrofit/retrofit.dart';
import 'package:dio/dio.dart';

part 'product_remote_datasource.g.dart';

@RestApi(baseUrl: "https://api.ecommerce.com/v1")
abstract class ProductRemoteDataSource {
  factory ProductRemoteDataSource(Dio dio, {String baseUrl}) = _ProductRemoteDataSource;

  @GET("/products")
  Future<ProductResponseModel> getProducts({
    @Query("page") required int page,
    @Query("limit") required int limit,
    @Query("category") String? category,
    @Query("q") String? searchQuery,
    @Query("min_price") double? minPrice,
    @Query("max_price") double? maxPrice,
    @Query("sort") String? sortBy,
  });

  @GET("/products/{id}")
  Future<ProductModel> getProductDetails(@Path("id") String id);

  @GET("/categories")
  Future<List<CategoryModel>> getCategories();
}

// Model classes
@JsonSerializable()
class ProductResponseModel {
  final List<ProductModel> data;
  final PaginationMetaModel meta;

  ProductResponseModel({required this.data, required this.meta});

  factory ProductResponseModel.fromJson(Map<String, dynamic> json) => 
      _$ProductResponseModelFromJson(json);
}

@JsonSerializable()
class PaginationMetaModel {
  final int currentPage;
  final int lastPage;
  final int total;
  final int perPage;

  PaginationMetaModel({
    required this.currentPage,
    required this.lastPage,
    required this.total,
    required this.perPage,
  });

  factory PaginationMetaModel.fromJson(Map<String, dynamic> json) => 
      _$PaginationMetaModelFromJson(json);
}
```

```dart
// lib/core/network/dio_client.dart
import 'package:dio/dio.dart';
import 'package:injectable/injectable.dart';

@module
abstract class NetworkModule {
  @singleton
  Dio get dio {
    final dio = Dio(BaseOptions(
      baseUrl: 'https://api.ecommerce.com/v1',
      connectTimeout: const Duration(seconds: 30),
      receiveTimeout: const Duration(seconds: 30),
      headers: {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
      },
    ));

    // Interceptors for auth, logging, retry
    dio.interceptors.addAll([
      AuthInterceptor(),
      LogInterceptor(requestBody: true, responseBody: true),
      RetryInterceptor(dio: dio),
    ]);

    return dio;
  }
}

class AuthInterceptor extends Interceptor {
  @override
  void onRequest(RequestOptions options, RequestInterceptorHandler handler) async {
    // Get token from secure storage
    final token = await SecureStorageService.read(key: 'auth_token');
    if (token != null) {
      options.headers['Authorization'] = 'Bearer $token';
    }
    handler.next(options);
  }

  @override
  void onError(DioException err, ErrorInterceptorHandler handler) {
    if (err.response?.statusCode == 401) {
      // Handle token refresh or logout
      eventBus.fire(UnauthorizedEvent());
    }
    handler.next(err);
  }
}
```

**Explanation:**

- **Retrofit**: Code generation creates the implementation of `ProductRemoteDataSource` from the abstract class with annotations.
- **Dio Interceptors**: Middleware for requests/responses. AuthInterceptor adds JWT tokens; RetryInterceptor handles network failures.
- **Pagination Meta**: API returns pagination info (current page, total pages) alongside data for infinite scroll implementation.

---

## **44.5 Presentation Layer: BLoC Implementation**

State management for the product catalog with pagination and filtering.

```dart
// lib/features/product/presentation/bloc/product_bloc.dart
import 'package:flutter_bloc/flutter_bloc.dart';
import 'package:freezed_annotation/freezed_annotation.dart';
import 'package:injectable/injectable.dart';

part 'product_bloc.freezed.dart';
part 'product_event.dart';
part 'product_state.dart';

@injectable
class ProductBloc extends Bloc<ProductEvent, ProductState> {
  final GetProducts getProducts;
  final GetCategories getCategories;

  ProductBloc({
    required this.getProducts,
    required this.getCategories,
  }) : super(const ProductState.initial()) {
    on<_LoadProducts>(_onLoadProducts);
    on<_LoadMore>(_onLoadMore);
    on<_FilterByCategory>(_onFilterByCategory);
    on<_Search>(_onSearch);
    on<_Sort>(_onSort);
  }

  Future<void> _onLoadProducts(
    _LoadProducts event,
    Emitter<ProductState> emit,
  ) async {
    emit(const ProductState.loading());
    
    final result = await getProducts(ProductParams(
      category: event.category,
      searchQuery: event.query,
    ));

    result.fold(
      (failure) => emit(ProductState.error(message: failure.message)),
      (response) => emit(ProductState.loaded(
        products: response.data,
        hasReachedMax: response.meta.currentPage >= response.meta.lastPage,
        currentPage: response.meta.currentPage,
        selectedCategory: event.category,
      )),
    );
  }

  Future<void> _onLoadMore(
    _LoadMore event,
    Emitter<ProductState> emit,
  ) async {
    final currentState = state;
    if (currentState is! _Loaded || currentState.hasReachedMax) return;

    emit(currentState.copyWith(isLoadingMore: true));

    final result = await getProducts(ProductParams(
      page: currentState.currentPage + 1,
      category: currentState.selectedCategory,
      searchQuery: currentState.searchQuery,
    ));

    result.fold(
      (failure) => emit(ProductState.error(message: failure.message)),
      (response) => emit(currentState.copyWith(
        products: [...currentState.products, ...response.data],
        hasReachedMax: response.meta.currentPage >= response.meta.lastPage,
        currentPage: response.meta.currentPage,
        isLoadingMore: false,
      )),
    );
  }
}

// State definitions
@freezed
class ProductState with _$ProductState {
  const factory ProductState.initial() = _Initial;
  const factory ProductState.loading() = _Loading;
  const factory ProductState.loaded({
    required List<Product> products,
    required bool hasReachedMax,
    required int currentPage,
    String? selectedCategory,
    String? searchQuery,
    @Default(false) bool isLoadingMore,
  }) = _Loaded;
  const factory ProductState.error({required String message}) = _Error;
}

// Event definitions
@freezed
class ProductEvent with _$ProductEvent {
  const factory ProductEvent.loadProducts({
    String? category,
    String? query,
  }) = _LoadProducts;
  const factory ProductEvent.loadMore() = _LoadMore;
  const factory ProductEvent.filterByCategory(String category) = _FilterByCategory;
  const factory ProductEvent.search(String query) = _Search;
  const factory ProductEvent.sort(String sortBy) = _Sort;
}
```

**Explanation:**

- **Freezed with BLoC**: Generates sealed classes for states and events, enabling exhaustive switch statements.
- **Pagination Logic**: `LoadMore` event fetches next page and appends to existing list. `hasReachedMax` prevents unnecessary requests.
- **Immutable State Updates**: Using `copyWith` to create new state instances with modified properties (BLoC best practice).
- **Error Handling**: Left side of Either converts to Error state, displayed in UI.

```dart
// lib/features/product/presentation/pages/product_list_page.dart
class ProductListPage extends StatelessWidget {
  const ProductListPage({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Products'),
        actions: [
          IconButton(
            icon: const Icon(Icons.shopping_cart),
            onPressed: () => context.router.push(const CartRoute()),
          ),
        ],
      ),
      body: Column(
        children: [
          // Search bar
          Padding(
            padding: const EdgeInsets.all(8.0),
            child: TextField(
              decoration: const InputDecoration(
                hintText: 'Search products...',
                prefixIcon: Icon(Icons.search),
              ),
              onChanged: (value) {
                context.read<ProductBloc>().add(ProductEvent.search(value));
              },
            ),
          ),
          
          // Category filter chips
          BlocBuilder<ProductBloc, ProductState>(
            buildWhen: (previous, current) => 
                previous.selectedCategory != current.selectedCategory,
            builder: (context, state) {
              return CategoryChips(
                selectedCategory: state.selectedCategory,
                onSelected: (category) {
                  context.read<ProductBloc>().add(
                    ProductEvent.filterByCategory(category),
                  );
                },
              );
            },
          ),
          
          // Product grid with pagination
          Expanded(
            child: BlocBuilder<ProductBloc, ProductState>(
              builder: (context, state) {
                return state.map(
                  initial: (_) => const Center(child: CircularProgressIndicator()),
                  loading: (_) => const Center(child: CircularProgressIndicator()),
                  error: (state) => ErrorWidget(message: state.message),
                  loaded: (state) => ProductGrid(
                    products: state.products,
                    hasReachedMax: state.hasReachedMax,
                    isLoadingMore: state.isLoadingMore,
                    onLoadMore: () {
                      context.read<ProductBloc>().add(const ProductEvent.loadMore());
                    },
                  ),
                );
              },
            ),
          ),
        ],
      ),
    );
  }
}

class ProductGrid extends StatelessWidget {
  final List<Product> products;
  final bool hasReachedMax;
  final bool isLoadingMore;
  final VoidCallback onLoadMore;

  const ProductGrid({
    super.key,
    required this.products,
    required this.hasReachedMax,
    required this.isLoadingMore,
    required this.onLoadMore,
  });

  @override
  Widget build(BuildContext context) {
    return NotificationListener<ScrollNotification>(
      onNotification: (notification) {
        if (notification is ScrollEndNotification &&
            notification.metrics.extentAfter == 0 &&
            !hasReachedMax &&
            !isLoadingMore) {
          onLoadMore();
        }
        return false;
      },
      child: GridView.builder(
        padding: const EdgeInsets.all(8),
        gridDelegate: const SliverGridDelegateWithFixedCrossAxisCount(
          crossAxisCount: 2,
          childAspectRatio: 0.75,
          crossAxisSpacing: 8,
          mainAxisSpacing: 8,
        ),
        itemCount: products.length + (isLoadingMore ? 1 : 0),
        itemBuilder: (context, index) {
          if (index >= products.length) {
            return const Center(child: CircularProgressIndicator());
          }
          return ProductCard(product: products[index]);
        },
      ),
    );
  }
}
```

**Explanation:**

- **BlocProvider/BlocBuilder**: Provider supplies the BLoC at the page level; Builder rebuilds specific widgets when state changes.
- **Pagination Trigger**: `NotificationListener` detects when user scrolls to bottom (`extentAfter == 0`), triggering LoadMore.
- **GridView**: Responsive grid layout for product cards. `childAspectRatio` controls card proportions.
- **buildWhen**: Optimization to prevent unnecessary rebuilds of CategoryChips when only product list changes.

---

## **44.6 Shopping Cart Implementation**

Managing cart state with local persistence using Hive.

```dart
// lib/features/cart/domain/entities/cart_item.dart
@freezed
class CartItem with _$CartItem {
  const factory CartItem({
    required String id,
    required Product product,
    required int quantity,
    required String? selectedSize,
    required String? selectedColor,
  }) = _CartItem;

  const CartItem._();

  double get totalPrice => product.price * quantity;
}

// lib/features/cart/data/datasources/cart_local_datasource.dart
@singleton
class CartLocalDataSource {
  final Box<CartItemModel> cartBox;

  CartLocalDataSource(this.cartBox);

  Future<List<CartItemModel>> getCartItems() async {
    return cartBox.values.toList();
  }

  Future<void> addToCart(CartItemModel item) async {
    // Check if item already exists with same variant
    final existingKey = cartBox.keys.firstWhere(
      (key) {
        final existing = cartBox.get(key);
        return existing?.productId == item.productId &&
               existing?.selectedSize == item.selectedSize &&
               existing?.selectedColor == item.selectedColor;
      },
      orElse: () => null,
    );

    if (existingKey != null) {
      final existing = cartBox.get(existingKey)!;
      await cartBox.put(
        existingKey,
        existing.copyWith(quantity: existing.quantity + item.quantity),
      );
    } else {
      await cartBox.add(item);
    }
  }

  Future<void> updateQuantity(String id, int quantity) async {
    final key = cartBox.keyAt(cartBox.values.toList().indexWhere(
      (item) => item.id == id,
    ));
    final item = cartBox.get(key);
    if (item != null) {
      await cartBox.put(key, item.copyWith(quantity: quantity));
    }
  }

  Future<void> removeFromCart(String id) async {
    final key = cartBox.keyAt(cartBox.values.toList().indexWhere(
      (item) => item.id == id,
    ));
    await cartBox.delete(key);
  }

  Future<void> clearCart() async {
    await cartBox.clear();
  }
}

// Hive adapter for CartItem
@HiveType(typeId: 1)
class CartItemModel extends HiveObject {
  @HiveField(0)
  final String id;
  
  @HiveField(1)
  final String productId;
  
  @HiveField(2)
  final int quantity;
  
  @HiveField(3)
  final String? selectedSize;
  
  @HiveField(4)
  final String? selectedColor;

  CartItemModel({
    required this.id,
    required this.productId,
    required this.quantity,
    this.selectedSize,
    this.selectedColor,
  });

  CartItemModel copyWith({int? quantity}) => CartItemModel(
        id: id,
        productId: productId,
        quantity: quantity ?? this.quantity,
        selectedSize: selectedSize,
        selectedColor: selectedColor,
      );
}
```

```dart
// lib/features/cart/presentation/bloc/cart_bloc.dart
@injectable
class CartBloc extends Bloc<CartEvent, CartState> {
  final CartRepository repository;

  CartBloc(this.repository) : super(const CartState.initial()) {
    on<_LoadCart>(_onLoadCart);
    on<_AddItem>(_onAddItem);
    on<_RemoveItem>(_onRemoveItem);
    on<_UpdateQuantity>(_onUpdateQuantity);
    on<_ClearCart>(_onClearCart);
  }

  Future<void> _onAddItem(_AddItem event, Emitter<CartState> emit) async {
    emit(const CartState.loading());
    
    final result = await repository.addToCart(
      product: event.product,
      quantity: event.quantity,
      size: event.size,
      color: event.color,
    );

    result.fold(
      (failure) => emit(CartState.error(failure.message)),
      (items) => emit(CartState.loaded(
        items: items,
        totalItems: _calculateTotalItems(items),
        totalPrice: _calculateTotalPrice(items),
      )),
    );
  }

  int _calculateTotalItems(List<CartItem> items) {
    return items.fold(0, (sum, item) => sum + item.quantity);
  }

  double _calculateTotalPrice(List<CartItem> items) {
    return items.fold(0.0, (sum, item) => sum + item.totalPrice);
  }
}

@freezed
class CartState with _$CartState {
  const factory CartState.initial() = _Initial;
  const factory CartState.loading() = _Loading;
  const factory CartState.loaded({
    required List<CartItem> items,
    required int totalItems,
    required double totalPrice,
  }) = _Loaded;
  const factory CartState.error(String message) = _Error;
}
```

**Explanation:**

- **Local Persistence**: Hive provides fast, encrypted local storage. Cart persists across app restarts.
- **Optimistic Updates**: UI updates immediately when user adds item, while background syncs with local storage.
- **Variant Handling**: Cart items distinguished by product ID + size + color combinations.
- **Calculated Properties**: `totalItems` and `totalPrice` computed from cart items for badge and summary displays.

---

## **44.7 Payment Integration (Stripe)**

Secure payment processing with PCI compliance (tokenization).

```dart
// lib/features/checkout/data/datasources/payment_datasource.dart
import 'package:flutter_stripe/flutter_stripe.dart';

@singleton
class StripePaymentDataSource {
  final Dio dio;

  StripePaymentDataSource(this.dio) {
    _init();
  }

  Future<void> _init() async {
    Stripe.publishableKey = 'pk_test_your_publishable_key';
    await Stripe.instance.applySettings();
  }

  Future<PaymentIntentModel> createPaymentIntent({
    required double amount,
    required String currency,
    required String customerId,
  }) async {
    try {
      final response = await dio.post(
        '/payments/create-intent',
        data: {
          'amount': (amount * 100).toInt(), // Convert to cents
          'currency': currency.toLowerCase(),
          'customer_id': customerId,
        },
      );

      return PaymentIntentModel.fromJson(response.data);
    } on DioException catch (e) {
      throw PaymentException(e.message ?? 'Failed to create payment intent');
    }
  }

  Future<void> presentPaymentSheet({
    required String clientSecret,
    required String ephemeralKey,
    required String customerId,
  }) async {
    try {
      // Initialize payment sheet
      await Stripe.instance.initPaymentSheet(
        paymentSheetParameters: SetupPaymentSheetParameters(
          paymentIntentClientSecret: clientSecret,
          customerEphemeralKeySecret: ephemeralKey,
          customerId: customerId,
          merchantDisplayName: 'E-Commerce Store',
          style: ThemeMode.system,
          billingDetails: const BillingDetails(
            // Pre-fill if available
          ),
        ),
      );

      // Present to user
      await Stripe.instance.presentPaymentSheet();
      
      // Confirm success
      await Stripe.instance.confirmPaymentSheetPayment();
    } on StripeException catch (e) {
      if (e.error.code == FailureCode.Canceled) {
        throw PaymentCanceledException();
      }
      throw PaymentException(e.error.message ?? 'Payment failed');
    }
  }
}
```

```dart
// lib/features/checkout/presentation/bloc/checkout_bloc.dart
@injectable
class CheckoutBloc extends Bloc<CheckoutEvent, CheckoutState> {
  final CreatePaymentIntent createPaymentIntent;
  final CreateOrder createOrder;
  final CartBloc cartBloc;

  CheckoutBloc({
    required this.createPaymentIntent,
    required this.createOrder,
    required this.cartBloc,
  }) : super(const CheckoutState.initial()) {
    on<_ProcessPayment>(_onProcessPayment);
  }

  Future<void> _onProcessPayment(
    _ProcessPayment event,
    Emitter<CheckoutState> emit,
  ) async {
    emit(const CheckoutState.processing());

    try {
      // 1. Create payment intent on server
      final intent = await createPaymentIntent(
        amount: event.totalAmount,
        currency: event.currency,
      );

      // 2. Present Stripe payment sheet
      await Stripe.instance.presentPaymentSheet(
        clientSecret: intent.clientSecret,
        ephemeralKey: intent.ephemeralKey,
        customerId: event.customerId,
      );

      // 3. Payment successful - create order
      final orderResult = await createOrder(
        items: cartBloc.state.items,
        paymentIntentId: intent.id,
        shippingAddress: event.shippingAddress,
      );

      orderResult.fold(
        (failure) => emit(CheckoutState.error(failure.message)),
        (order) {
          cartBloc.add(const CartEvent.clearCart());
          emit(CheckoutState.success(order: order));
        },
      );

    } on PaymentCanceledException {
      emit(const CheckoutState.canceled());
    } on PaymentException catch (e) {
      emit(CheckoutState.error(e.message));
    }
  }
}
```

**Explanation:**

- **Tokenization**: Stripe SDK collects card details and returns a token. Your server never sees raw card numbers (PCI compliance).
- **Payment Sheet**: Pre-built UI for card entry, handling validation, 3D Secure, and error states.
- **Server-Side Intent**: PaymentIntent created on backend with amount; client only receives client_secret to confirm.
- **Order Creation**: Only create order after successful payment confirmation to prevent inventory issues.

---

## **44.8 Order Tracking with Push Notifications**

Real-time order updates using Firebase Cloud Messaging.

```dart
// lib/features/order/domain/entities/order.dart
@freezed
class Order with _$Order {
  const factory Order({
    required String id,
    required List<CartItem> items,
    required double totalAmount,
    required OrderStatus status,
    required DateTime createdAt,
    required ShippingAddress shippingAddress,
    String? trackingNumber,
    DateTime? estimatedDelivery,
  }) = _Order;
}

enum OrderStatus {
  pending,
  confirmed,
  processing,
  shipped,
  outForDelivery,
  delivered,
  cancelled,
}

// lib/services/notification_service.dart
@singleton
class NotificationService {
  final FirebaseMessaging _messaging = FirebaseMessaging.instance;
  final FlutterLocalNotificationsPlugin _localNotifications = 
      FlutterLocalNotificationsPlugin();

  Future<void> initialize() async {
    // Request permissions
    await _messaging.requestPermission(
      alert: true,
      badge: true,
      sound: true,
    );

    // Configure foreground presentation
    await FirebaseMessaging.instance.setForegroundNotificationPresentationOptions(
      alert: true,
      badge: true,
      sound: true,
    );

    // Handle FCM messages
    FirebaseMessaging.onMessage.listen(_handleForegroundMessage);
    FirebaseMessaging.onMessageOpenedApp.listen(_handleBackgroundMessage);
    
    // Get initial message (app opened from notification)
    final initialMessage = await _messaging.getInitialMessage();
    if (initialMessage != null) {
      _handleNotificationNavigation(initialMessage.data);
    }
  }

  void _handleForegroundMessage(RemoteMessage message) {
    final notification = message.notification;
    final data = message.data;

    if (notification != null) {
      // Show local notification
      _localNotifications.show(
        notification.hashCode,
        notification.title,
        notification.body,
        NotificationDetails(
          android: AndroidNotificationDetails(
            'order_updates',
            'Order Updates',
            channelDescription: 'Updates about your orders',
            importance: Importance.high,
            priority: Priority.high,
          ),
          iOS: DarwinNotificationDetails(),
        ),
        payload: jsonEncode(data),
      );

      // Update order bloc if order update
      if (data['type'] == 'order_update') {
        getIt<OrderBloc>().add(OrderEvent.statusUpdated(
          orderId: data['order_id'],
          status: data['status'],
        ));
      }
    }
  }

  void _handleNotificationNavigation(Map<String, dynamic> data) {
    final type = data['type'];
    final orderId = data['order_id'];

    if (type == 'order_update' && orderId != null) {
      // Navigate to order details
      getIt<AppRouter>().push(OrderDetailsRoute(orderId: orderId));
    }
  }

  Future<String?> getToken() async {
    return await _messaging.getToken();
  }
}
```

**Explanation:**

- **Firebase Messaging**: Handles push notifications from server when order status changes.
- **Local Notifications**: Display notifications when app is in foreground (system doesn't show them by default).
- **Deep Linking**: Navigate to specific order when user taps notification.
- **Bloc Integration**: Automatically update order status in UI when push received, even if app is open.

---

## **Chapter Summary**

In this chapter, we built a production-ready e-commerce application demonstrating:

### **Key Takeaways:**

1. **Clean Architecture**: Separated concerns into Domain (business logic), Data (API/storage), and Presentation (UI/BLoC). Enables testing and framework independence.

2. **BLoC Pattern**: Managed complex state (loading, error, loaded, pagination) with immutable states and explicit events. Used `freezed` for type-safe state management.

3. **Pagination**: Implemented infinite scroll with `NotificationListener` detecting scroll position, triggering `LoadMore` events when reaching end of list.

4. **Local Persistence**: Used Hive for cart storage with custom adapters for complex objects. Cart survives app restarts and device reboots.

5. **Payment Security**: Integrated Stripe with tokenization ensuring PCI compliance. Payment sheet handles sensitive data collection; backend creates PaymentIntents.

6. **Real-time Updates**: Firebase Cloud Messaging for order status notifications, with automatic UI updates via BLoC event injection.

### **Architecture Benefits:**
- ✅ **Testability**: Use cases and repositories are pure Dart, easily unit tested
- ✅ **Scalability**: New features added as modules without touching existing code
- ✅ **Maintainability**: Clear separation makes debugging and refactoring safer
- ✅ **Platform Agnostic**: Domain layer knows nothing about Flutter, HTTP, or databases

---

## **Next Steps**

In the next chapter, **Chapter 45: Project 2 - Social Media Feed**, we will build a real-time social media application featuring infinite scrolling feeds with pagination, image/video handling and caching, real-time updates via WebSockets, like/comment functionality with optimistic updates, and comprehensive push notification handling for social interactions.

---

**End of Chapter 44**